In [1]:
import os
import geopandas as gpd 
import pandas as pd
from datetime import datetime
import string
import random


#WCA data processing 

In [2]:
ref_columns = ['obs', 'p-obs', 'calvingyr', 'calvingloc', 'area', 'perimeter', 'length', 'lon', 'lat', 'geometry',
                'scenedate', 'imgref', 'mothercert', 'shpcert', 'georef', 'ddinfo', 'sensor', 'beam_mode', 'pol']


In [3]:
# Read the CSA files in pandas dataframe
file_path = r'3020.shp'

filename = os.path.basename(file_path)[:-4]

raw_gdf = gpd.read_file(file_path)


In [4]:
# filter the data based on the ICE TYPE value i = {}, ii = {}
filtered_ice_ty = raw_gdf[raw_gdf['ICETYPE']=='ii']

#added new reference colums to the the filtered data
new_data =  pd.concat([filtered_ice_ty,pd.DataFrame(columns=ref_columns)])

C:\Users\shubh\AppData\Local\Temp\ipykernel_13480\1705199530.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_data =  pd.concat([filtered_ice_ty,pd.DataFrame(columns=ref_columns)])


In [5]:
new_data.reset_index(drop=True, inplace=True)

In [6]:
# add the values to the new columns
new_data['INSTID'] = new_data['INSTID'].astype('int')
new_data.sort_values(by=['INSTID'],inplace=True)

new_data = new_data[new_data['INSTID'].notna()]
print(len(new_data))

126


In [7]:
date_format = "%Y%m%d_%H%M%S" 

new_data['obs'] = new_data['IMGREF'].apply(lambda x: f"{x[:18]}_0_{''.join(random.sample(string.ascii_uppercase, 3))}")
new_data['scenedate'] = new_data['IMGREF'].apply(lambda x: x[:15])
new_data['calvingyr'] = new_data['IMGREF'].apply(lambda x: datetime.strptime(x[:15],date_format).year)
new_data['pol'] = new_data['IMGREF'].apply(lambda x: x[24:26])
new_data['sensor'] = new_data['IMGREF'].apply(lambda x: x[16:18])
new_data['beam_mode'] = new_data['IMGREF'].apply(lambda x: x[19:23].replace("_", ""))
new_data['imgref'] = new_data['IMGREF']
new_data['ddinfo'] = 'drifting'
new_data['mothercert'] = 1
new_data['shpcert'] = 1
new_data['calvingloc'] = 'WH'

In [8]:
new_data.columns

Index(['INSTID', 'OBJ', 'FNAME', 'ICETYPE', 'ICETYPE2', 'IMGREF', 'TARGET',
       'VALIDATE', 'TARGETOBJ', 'FROMDATE', 'TODATE', 'NOTES', 'geometry',
       'obs', 'p-obs', 'calvingyr', 'calvingloc', 'area', 'perimeter',
       'length', 'lon', 'lat', 'scenedate', 'imgref', 'mothercert', 'shpcert',
       'georef', 'ddinfo', 'sensor', 'beam_mode', 'pol'],
      dtype='object')

In [9]:
new_data.loc[0, 'p-obs'] = new_data.loc[0, 'obs']

for i in range(1, len(new_data)):
    new_data.loc[i, 'p-obs'] = new_data.loc[i-1, 'obs']

In [10]:
# Select the ones you want
ref_columns.append('NOTES')
formatted_csa = new_data[ref_columns]




In [11]:
ref_columns

['obs',
 'p-obs',
 'calvingyr',
 'calvingloc',
 'area',
 'perimeter',
 'length',
 'lon',
 'lat',
 'geometry',
 'scenedate',
 'imgref',
 'mothercert',
 'shpcert',
 'georef',
 'ddinfo',
 'sensor',
 'beam_mode',
 'pol',
 'NOTES']

In [12]:
formatted_csa.to_file(rf'D:\UNI_Stir\Projects\CI2D3\InProgress\python\{filename}_formatted.shp', driver='ESRI Shapefile')